### Import

In [1]:
from main import *
from gen_image import *
from ptp_inversion import load_512

/home/dingkaixin/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Init models

In [3]:
model_path = "../../s"
generator, ldm_stable, device = init_model(model_path)
text_encoder = CLIPTextModel.from_pretrained(model_path, subfolder="text_encoder",
                                                 revision=False)
text_encoder = text_encoder.to(device)
tokenizer = MultiTokenCLIPTokenizer.from_pretrained(model_path, subfolder="tokenizer")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CLIPTokenizer'. 
The class this function is called from is 'MultiTokenCLIPTokenizer'.
/home/dingkaixin/miniconda3/envs/ldm/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'dropout': 0.0} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.
/home/dingkaixin/miniconda3/envs/ldm/lib/python3.8/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:101: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_

#### Output Dir

In [4]:

import datetime
from ptp_inversion import Inversion, make_controller
import ptp_utils
inversion = Inversion(ldm_stable)
inversion.init_prompt("")

def get_image_file(path):
    img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    img_files = [f for f in os.listdir(path) if f.lower().endswith(img_extensions)]
    return img_files[0]
src_embedding_dir_path = "dog_hat"
ref_embedding_dir_path = "cat_hat"
embedding_dir_path = src_embedding_dir_path + "-" + ref_embedding_dir_path
_src_file_name = "dog_hat"
_ref_file_name = "cat_hat"
_image_dir = "./examples"
steps = 1000
recaption = False
if steps == 1000:
    if recaption is True:
        ref_embedding_dir = "./output/" + ref_embedding_dir_path + "/dog_hat_05_09_2024_0715_recaption/"
        src_embedding_dir = "./output/" + src_embedding_dir_path + "/dog_05_09_2024_0009_recaption/"
        # ref_embedding_dir = "./output/" + ref_embedding_dir_path + "/real-cat-mouse_05_09_2024_0827_recaption/"
        # src_embedding_dir = "./output/" + src_embedding_dir_path + "/real-cat_05_09_2024_0750_recaption/"
    else:
        ref_embedding_dir = "./output/" + ref_embedding_dir_path + "/cat_hat_05_08_2024_2045/"
        src_embedding_dir = "./output/" + src_embedding_dir_path + "/dog_hat_05_08_2024_2000/"

        # ref_embedding_dir = "./output/" + ref_embedding_dir_path + "/real-cat-mouse_05_08_2024_1124/"
        # src_embedding_dir = "./output/" + src_embedding_dir_path + "/real-cat_05_08_2024_1102/"
else:
    ref_embedding_dir = "./output/" + ref_embedding_dir_path + "/real-cat-mouse_04_27_2024_2325/"
    src_embedding_dir = "./output/" + src_embedding_dir_path + "/real-cat_04_27_2024_2318/"
ref_file_name = get_image_file(os.path.join(_image_dir, _ref_file_name)).split('.')[0]
src_file_name = get_image_file(os.path.join(_image_dir, _src_file_name)).split('.')[0]


src_embeddings = torch.load(os.path.join(src_embedding_dir, str(steps) + ".bin"))
ref_embeddings = torch.load(os.path.join(ref_embedding_dir, str(steps) + ".bin"))

uncond_input = tokenizer(
            [""], padding="max_length",
            return_tensors="pt"
        )


def gen_image(
        output_img_path_name,
        src_image_path_name,
        tar_image_path_name,
        src_embedding,
        ref_embedding,
        null_embeddings,
        num_inference_steps,
        cfg_guidence=2.0,
        tar_weight=2.0,
        src_weight=2.0,
        save_all=False,
):
    print("loading image", src_image_path_name)
    tar_image = load_512(tar_image_path_name)
    src_image = load_512(src_image_path_name)
    src_latent = inversion.image2latent(src_image)
    tar_latent = inversion.image2latent(tar_image)

    
    # generate picture
    latents = inversion.backward_diffusion_ptp(
        src_latent,
        tar_latent,
        src_embedding,
        ref_embedding,
        null_embeddings,
        num_inference_steps,
        cfg_guidence,
        tar_weight,
        src_weight,
        save_all,
    )
    print("latents = ", latents.shape)
    if save_all:
        for i, latent in enumerate(latents):
            images = ptp_utils.latent2image(ldm_stable.vae, latent.detach())
            output_img_list = [Image.fromarray(images[0, :, :, :])]
            output_img_list[0].save(output_img_path_name[:-4] + "_" + str(i) + ".png")
    else:
        # save picture
        images = ptp_utils.latent2image(ldm_stable.vae, latents.detach())
        print("saving image", output_img_path_name)
        output_img_list = [Image.fromarray(images[1, :, :, :])]
        output_img_list[0].save(output_img_path_name)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:34                                                                                   │
│                                                                                                  │
│   31 else:                                                                                       │
│   32 │   ref_embedding_dir = "./output/" + ref_embedding_dir_path + "/real-cat-mouse_04_27_20    │
│   33 │   src_embedding_dir = "./output/" + src_embedding_dir_path + "/real-cat_04_27_2024_231    │
│ ❱ 34 ref_file_name = get_image_file(os.path.join(_image_dir, _ref_file_name)).split('.')[0]      │
│   35 src_file_name = get_image_file(os.path.join(_image_dir, _src_file_name)).split('.')[0]      │
│   36                                                                                             │
│   37                                                                                             │
│                                                                                                  │
│ in get_image_file:9                                                                              │
│                                                                                                  │
│    6                                                                                             │
│    7 def get_image_file(path):                                                                   │
│    8 │   img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')                              │
│ ❱  9 │   img_files = [f for f in os.listdir(path) if f.lower().endswith(img_extensions)]         │
│   10 │   return img_files[0]                                                                     │
│   11 src_embedding_dir_path = "dog_hat"                                                          │
│   12 ref_embedding_dir_path = "cat_hat"                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '../examples/cat_hat'

#### Gen Image

In [ ]:
null_embeddings = text_encoder(uncond_input.input_ids.to(device))[0]
# output_img_dir = "output_images/5-8cfgtrace-" + embedding_dir_path + "_" + str(steps) + "/"
output_img_dir = "output_images/5-9new" + embedding_dir_path + "_" + str(steps) + "/"
now = datetime.datetime.now()
time_smp = str(now.day) + "-" + str(now.hour) + "-" + str(now.minute) + "-" + str(now.second) 
os.makedirs(output_img_dir, exist_ok=True)
test = True
src_image_path_name = os.path.join(_image_dir, _src_file_name) + "/" + get_image_file(os.path.join(_image_dir, _src_file_name))
ref_image_path_name = os.path.join(_image_dir, _ref_file_name) + "/" + get_image_file(os.path.join(_image_dir, _ref_file_name))

if test:    
    gen_image(
                output_img_dir + "DVCT" + "_" + time_smp + "_" + ref_file_name + "_" + src_file_name \
                + "-n" + str(50) + "-cfg" + str(1) + ".png",
                src_image_path_name,
                ref_image_path_name,
                src_embeddings,
                ref_embeddings,
                null_embeddings=null_embeddings,
                num_inference_steps=20,
                cfg_guidence= None,
                src_weight=None,
                tar_weight=1,
                save_all=False,
            )
else:
    for i in range (20, 100, 30):
        for j in range(1, 5, 1):
            gen_image(
                output_img_dir + "DVCT" + "_" + time_smp + "_" + ref_file_name + "_" + src_file_name \
                + "-n" + str(i) + "-weight" + str(j / 2) + ".png",
                src_image_path_name,
                ref_image_path_name,
                src_embeddings,
                ref_embeddings,
                null_embeddings=null_embeddings,
                num_inference_steps=i,
                cfg_guidence= None,
                weight=j * 1.0/2,
                save_all=False,
            )

loading image ../examples/dog_hat/dog_hat.jpg


100%|██████████| 20/20 [00:03<00:00,  5.37it/s]


latents =  torch.Size([2, 4, 64, 64])
saving image output_images/5-9newdog_hat-cat_hat_1000/DVCT_9-17-17-14_cat_hat_dog_hat-n50-cfg1.png


#### Gen Image with DIRECT Inversion & Inversion Free